In [10]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

output_directory = '/home/marco/DataspellProjects/thesis/output/svm_rbf/'

palette = sns.color_palette(["#2176AB", "#F97662", "#FFBF00", "#50C878", "#B284BE"])
sns.set(style="whitegrid", font_scale=1.5)
sns.set_palette(palette)

df = pd.read_csv('../../../data/ST002498_z.csv')

In [11]:
#import random forest model
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

X = df.drop(columns=['Diagnosis'])
y = df['Diagnosis']

#

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

space = {
    'C': hp.loguniform('C', np.log(0.001), np.log(1000)), 
    'kernel': hp.choice('kernel', ['linear', 'rbf', 'poly']),  
    'degree': hp.quniform('degree', 2, 5, 1),  
    'gamma': hp.loguniform('gamma', np.log(0.0001), np.log(10)),
    'probability': True 
}

def objective(space):
    clf = SVC(C=space['C'], kernel=space['kernel'], degree=int(space['degree']), gamma=space['gamma'], probability=True)

    clf.fit(X_train, y_train)
    pred = clf.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, pred)
    return {'loss': -auc, 'status': STATUS_OK}

trials = Trials()

best_hyperparams = fmin(fn=objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=1000,
                        trials=trials)

best_params = space_eval(space, best_hyperparams)

best_clf = SVC(C=best_params['C'], kernel=best_params['kernel'], degree=int(best_params['degree']), gamma=best_params['gamma'], probability=True)

best_clf.fit(X_train, y_train)

pred = best_clf.predict_proba(X_test)[:, 1]

best_auc = roc_auc_score(y_test, pred)

# Stampa del miglior score AUC ottenuto
print("Miglior score AUC:", best_auc)


100%|██████████| 1000/1000 [01:28<00:00, 11.24trial/s, best loss: -0.6122727272727273]
Miglior score AUC: 0.6013068181818182
